In [116]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.data_preprocessing import datetime_processing, userinput_processing, holiday_processing, create_x, create_y

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, precision_score, recall_score, accuracy_score

In [117]:
import os
os.chdir('/Users/varunvenkatesh/Documents/Github/ev_charging')
os.getcwd()

'/Users/varunvenkatesh/Documents/GitHub/ev_charging'

In [118]:
df = pd.read_parquet('data/sessions/caltech/').reset_index(drop=True)
df = datetime_processing(df)
df = userinput_processing(df)
df = holiday_processing(df)

In [119]:
df.head()

,_id,sessionID,stationID,spaceID,siteID,clusterID,connectionTime,disconnectTime,kWhDelivered,doneChargingTime,...,doneChargingTimeDay,userID,milesRequested,WhPerMile,minutesAvailable,modifiedAt,paymentRequired,requestedDeparture,kWhRequested,is_holiday
0,5bc9160af9af8b0dad3c0690,2_39_95_27_2018-05-01 13:23:18.374684,2-39-95-27,CA-319,2,39,2018-05-01 13:23:18+00:00,2018-05-02 01:53:41+00:00,13.410,2018-05-01 17:17:07+00:00,...,1.0,NaN,NaN,NaN,NaN,None,None,None,NaN,False
1,5bc9160af9af8b0dad3c0691,2_39_79_380_2018-05-01 13:54:33.599590,2-39-79-380,CA-489,2,39,2018-05-01 13:54:34+00:00,2018-05-01 23:15:35+00:00,8.618,2018-05-01 15:35:39+00:00,...,1.0,NaN,NaN,NaN,NaN,None,None,None,NaN,False
2,5bc9160af9af8b0dad3c0692,2_39_139_28_2018-05-01 14:46:04.441389,2-39-139-28,CA-303,2,39,2018-05-01 14:46:04+00:00,2018-05-02 00:54:07+00:00,13.969,2018-05-01 23:57:02+00:00,...,1.0,NaN,NaN,NaN,NaN,None,None,None,NaN,False
3,5bc9160af9af8b0dad3c0693,2_39_79_378_2018-05-01 14:56:28.908623,2-39-79-378,CA-326,2,39,2018-05-01 14:56:29+00:00,2018-05-01 21:52:45+00:00,6.914,2018-05-01 16:07:56+00:00,...,1.0,NaN,NaN,NaN,NaN,None,None,None,NaN,False
4,5bc9160af9af8b0dad3c0694,2_39_88_24_2018-05-01 15:14:50.443137,2-39-88-24,CA-314,2,39,2018-05-01 15:14:50+00:00,2018-05-01 22:51:30+00:00,13.785,2018-05-01 20:18:12+00:00,...,1.0,NaN,NaN,NaN,NaN,None,None,None,NaN,False


In [120]:
def get_results(y_test, prediction):
    cm = confusion_matrix(y_test,prediction)
    make_classification_plot(cm)
    
    results = {'tpr': cm[1, 1]/np.sum(cm[1]),
               'fpr': cm[0,1]/np.sum(cm[0]),
               'accuracy': accuracy_score(y_test, prediction),
               'precision': precision_score(y_test, prediction),
               'recall': recall_score(y_test, prediction),
        'f1':f1_score(y_test,prediction)}
    return results

In [121]:
def make_classification_plot(cm):
    disp = ConfusionMatrixDisplay(cm)
    disp = disp.plot(include_values=True, cmap='viridis', ax=None, xticks_rotation='horizontal')
    plt.grid(False)
    plt.show()

In [122]:
results = {}

In [126]:
# create x and y 
start = '2018-05-01'
end = '2021-09-13'
spaceID = 'CA-303'
caiso_fp = 'data/caiso_dataset_5min.csv'

In [127]:
X = create_x(start, end, caiso_fp)
y = create_y(df, start, end, spaceID)

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'RangeIndex'

In [ ]:
#tmp = df.copy()
#tmp = tmp[tmp['spaceID'] == 'CA-303'].sort_index()

In [ ]:
#y = pd.DataFrame(index=pd.date_range('2018-05-1','2021-09-13', inclusive='both', freq='h', tz=0),columns=['is_available','sessionID'])
#y['is_available'] = 1
#tmp.reset_index(inplace=True)
#for i in list(tmp.index):
#    start_ = tmp.loc[i,'connectionTime']
#    end_ = tmp.loc[i,'disconnectTime'] 
#    session_ = tmp.loc[i,'sessionID']
#    y.loc[start_:end_,['is_available','sessionID']] = 0, session_

In [ ]:
y.drop(columns='sessionID', inplace=True)

In [ ]:
#y['is_available'].value_counts()

In [ ]:
#caiso_df = pd.read_csv('data/caiso_dataset_5min.csv')
#caiso_df['datetime'] = pd.to_datetime(caiso_df['date'] + ' ' + caiso_df['Time'], utc=True)
#caiso_df.info()

In [ ]:
#caiso_df = caiso_df.set_index('datetime')
#caiso_hourly = caiso_df.groupby(pd.Grouper(freq='1h')).mean()
#caiso_hourly.index.tz_localize(None)
#caiso_hourly.head()

In [ ]:
#print(len(caiso_hourly))

In [ ]:
#X = pd.DataFrame(index=pd.date_range('2018-05-1','2021-09-13', inclusive='both', freq='h', tz=0),columns=['dow','hour','month'])
#X = X.join(caiso_hourly)

In [ ]:
X.head()

In [ ]:
#X['dow'] = X.index.dayofweek
#X['hour'] = X.index.hour
#X['month'] = X.index.month
#X['connectionTime'] = X.index
#X = holiday_processing(X).drop(columns=['connectionTime'])
#X.head()

In [ ]:
print(f'charger CA-303 is available {np.round(y.mean()*100,3)}% of the time')

In [ ]:
X.fillna(0.0, inplace=True)

In [ ]:
X.info()

In [ ]:
X.shape[0] == y.shape[0]

# LSTM

In [ ]:
# Create hold out test set
bin_numbers = pd.qcut(y['is_available'], q=10, labels=False, duplicates='drop')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, stratify=bin_numbers, random_state=42)
print(f'the training data has an average availability of {np.round(y_train.mean()*100,3)}%')

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
tf.random.set_seed(42)

In [ ]:
#scaler = MinMaxScaler(feature_range=(0,1))
look_back = 1

In [ ]:
model = Sequential()
model.add(LSTM(4, input_shape=(1,39)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
X_train = np.reshape(X_train.values.astype(float), (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test.values.astype(float), (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
[print(i.shape, i.dtype) for i in model.inputs]
[print(o.shape, o.dtype) for o in model.outputs]
[print(l.name, l.input_shape, l.dtype) for l in model.layers]

In [ ]:
X_train.shape

In [ ]:
y_train = y_train.drop('sessionID', axis=1) 
y_test = y_test.drop('sessionID', axis=1)

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=8, verbose=2)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
# MSE with CAISO
score = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {score}')

In [ ]:
# MSE without CAISO
score = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {score}')